<a href="https://colab.research.google.com/github/zanqi/tiny_asr/blob/main/asr_gap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get update && sudo apt-get install -y libsndfile1 ffmpeg
!pip install cython
!pip install nemo_toolkit['asr']

!pip install transformers datasets accelerate soundfile librosa jiwer
!pip install --upgrade lhotse

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,978 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

  Using cached lhotse-1.32.2-py3-none-any.whl.metadata (19 kB)
Using cached lhotse-1.32.2-py3-none-any.whl (889 kB)
  Attempting uninstall: lhotse
    Found existing installation: lhotse 1.31.1
    Uninstalling lhotse-1.31.1:
      Successfully uninstalled lhotse-1.31.1


In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from jiwer import wer
import nemo.collections.asr as nemo_asr
from transformers import pipeline

NUM_SAMPLES = 50
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Running on: {DEVICE}")
print(f"Sample limit: {NUM_SAMPLES}")

[NeMo W 2026-02-20 03:49:43 nemo_logging:405] Megatron num_microbatches_calculator not found, using Apex version.


Running on: cuda
Sample limit: 50


In [ ]:
print("Loading LibriSpeech test-clean...")

def load_librispeech(subset):
  dataset = load_dataset("openslr/librispeech_asr", subset, split="test", streaming=True)
  dataset = dataset.take(NUM_SAMPLES)
  references = []
  file_paths = []

  import os
  import soundfile as sf
  import IPython.display as ipd

  os.makedirs("temp_audio", exist_ok=True)

  for i, sample in enumerate(dataset):
    text = sample['text']
    audio_array = sample['audio']['array']
    sample_rate = sample['audio']['sampling_rate']

    temp_path = f"temp_audio/sample_{i}.wav"
    sf.write(temp_path, audio_array, sample_rate)

    references.append(text)
    file_paths.append(temp_path)

  return dataset, references, file_paths

dataset, references, file_paths = load_librispeech("clean")

print(f"Loaded {len(references)} samples.")
print(f"ref: {references[0]}")
ipd.Audio(file_paths[0])

Loading LibriSpeech test-clean...


Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Loaded 50 samples.
ref: CONCORD RETURNED TO ITS PLACE AMIDST THE TENTS


In [ ]:
def evaluate_nemo(model_name):
  asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name=model_name)
  asr_model.to(DEVICE)
  asr_model.freeze()

  predictions = asr_model.transcribe(file_paths)

  norm_preds = []
  for p in predictions:
      if isinstance(p, str):
          norm_preds.append(p.upper())
      else:
          norm_preds.append(p.text.upper())

  error_rate = wer(references, norm_preds)

  del asr_model
  torch.cuda.empty_cache()
  return error_rate

nemo_small_wer = evaluate_nemo("nvidia/stt_en_conformer_ctc_small")
nemo_large_wer = evaluate_nemo("nvidia/stt_en_conformer_ctc_large")

print(f"\nNemo Small WER: {nemo_small_wer}")
print(f"Nemo Large WER: {nemo_large_wer}")

[NeMo I 2026-02-20 04:40:57 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2026-02-20 04:40:57 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2026-02-20 04:40:57 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librisp

[NeMo I 2026-02-20 04:40:57 nemo_logging:393] PADDING: 0
[NeMo I 2026-02-20 04:40:59 nemo_logging:393] Model EncDecCTCModelBPE was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_en_conformer_ctc_small/snapshots/e5b9941cc1b0b8a08c29b31a111c674f3040a80f/stt_en_conformer_ctc_small.nemo.


[NeMo W 2026-02-20 04:40:59 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: use_start_end_token
[NeMo W 2026-02-20 04:40:59 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 13it [00:01, 10.57it/s]


[NeMo I 2026-02-20 04:41:22 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2026-02-20 04:41:22 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2026-02-20 04:41:22 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librisp

[NeMo I 2026-02-20 04:41:22 nemo_logging:393] PADDING: 0
[NeMo I 2026-02-20 04:41:27 nemo_logging:393] Model EncDecCTCModelBPE was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_en_conformer_ctc_large/snapshots/5f18b90411b9fdfc9b52b8caf636f204491afb01/stt_en_conformer_ctc_large.nemo.


[NeMo W 2026-02-20 04:41:28 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: use_start_end_token
[NeMo W 2026-02-20 04:41:28 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 13it [00:02,  4.88it/s]



Nemo Small WER: 0.03934426229508197
Nemo Large WER: 0.018579234972677595


In [ ]:
dataset, references, file_paths = load_librispeech("other")

nemo_small_wer = evaluate_nemo("nvidia/stt_en_conformer_ctc_small")
nemo_large_wer = evaluate_nemo("nvidia/stt_en_conformer_ctc_large")

print(f"\nNemo Small WER: {nemo_small_wer}")
print(f"Nemo Large WER: {nemo_large_wer}")

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

[NeMo I 2026-02-20 04:42:56 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2026-02-20 04:42:57 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2026-02-20 04:42:57 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librisp

[NeMo I 2026-02-20 04:42:57 nemo_logging:393] PADDING: 0
[NeMo I 2026-02-20 04:43:00 nemo_logging:393] Model EncDecCTCModelBPE was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_en_conformer_ctc_small/snapshots/e5b9941cc1b0b8a08c29b31a111c674f3040a80f/stt_en_conformer_ctc_small.nemo.


...

[NeMo I 2026-02-20 04:43:20 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2026-02-20 04:43:20 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2026-02-20 04:43:20 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librisp

[NeMo I 2026-02-20 04:43:20 nemo_logging:393] PADDING: 0
[NeMo I 2026-02-20 04:43:22 nemo_logging:393] Model EncDecCTCModelBPE was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_en_conformer_ctc_large/snapshots/5f18b90411b9fdfc9b52b8caf636f204491afb01/stt_en_conformer_ctc_large.nemo.


[NeMo W 2026-02-20 04:43:22 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: use_start_end_token
[NeMo W 2026-02-20 04:43:22 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 13it [00:03,  4.16it/s]


Nemo Small WER: 0.053631284916201116
Nemo Large WER: 0.03128491620111732


In [ ]:
dataset, references, file_paths = load_librispeech("other")

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
def evaluate_whisper(model_name):
  pipe = pipeline("automatic-speech-recognition",
                  model=model_name,
                  device=DEVICE, chunk_length_s=30)
  results = pipe(file_paths)
  predictions = [r["text"].upper() for r in results]

  import re
  clean_preds = [re.sub(r'[^\w\s]', '', p) for p in predictions]

  print(f"ref: {references[0]}")
  print(f"pred: {clean_preds[0]}")

  error_rate = wer(references, clean_preds)

  del pipe
  torch.cuda.empty_cache()

  return error_rate

# whisper_tiny_wer = evaluate_whisper("openai/whisper-tiny")
whisper_large_wer = evaluate_whisper("openai/whisper-large-v3")

# print(f"\nWhisper Tiny WER: {whisper_tiny_wer}")
print(f"Whisper Large WER: {whisper_large_wer}")

Device set to use cuda
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


ref: I AM FROM THE CUTTER LYING OFF THE COAST
pred:  IM FROM THE CUTTER LYING OFF THE COAST
Whisper Large WER: 0.060335195530726256


### clean
Nemo Small WER: 0.03934426229508197 \
Nemo Large WER: 0.018579234972677595 \
Whisper Tiny WER: 0.07650273224043716 \
Whisper Large WER: 0.04590163934426229

### other
Nemo Small WER: 0.053631284916201116 \
Nemo Large WER: 0.03128491620111732 \
Whisper Tiny WER: 0.1329608938547486 \
Whisper Large WER: 0.060335195530726256


### Open ASR benchmark results for Moonshine (WER %)

| Dataset               | Tiny (34M) | Small (123M) | Medium (245M) |
|:----------------------|----------:|-------------:|--------------:|
| AMI                   | 19.03     | 12.54        | 10.68         |
| Earnings-22           | 20.27     | 13.53        | 11.90         |
| GigaSpeech            | 13.90     | 10.41        | 9.46          |
| LibriSpeech (clean)   | 4.49      | 2.49         | 2.08          |
| LibriSpeech (other)   | 12.09     | 6.78         | 5.00          |
| SPGISpeech            | 6.16      | 3.19         | 2.58          |
| TED-LIUM              | 6.12      | 3.77         | 2.99          |
| VoxPopuli             | 14.02     | 9.98         | 8.54          |
| **Average**           | **12.01** | **7.84**     | **6.65**      |